In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33039
2,Huelva,Confirmados PDIA 14 días,1100
3,Huelva,Tasa PDIA 14 días,"214,35391780501587"
4,Huelva,Confirmados PDIA 7 días,460
5,Huelva,Total Confirmados,33222
6,Huelva,Curados,31154
7,Huelva,Fallecidos,389


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33039.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9044.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 817 personas en los últimos 7 días 

Un positivo PCR cada 315 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33039.0,460.0,1100.0,513170.0,89.638911,214.353918,53.0
Huelva-Costa,19035.0,255.0,647.0,289548.0,88.068300,223.451725,29.0
Condado-Campiña,10734.0,168.0,357.0,156231.0,107.533076,228.507787,24.0
Huelva (capital),9044.0,176.0,457.0,143837.0,122.360728,317.720753,23.0
San Juan del Puerto,572.0,18.0,48.0,9411.0,191.265540,510.041441,6.0
Moguer,1383.0,28.0,70.0,21867.0,128.046829,320.117071,5.0
Palos de la Frontera,712.0,13.0,29.0,11742.0,110.713677,246.976665,5.0
Cartaya,1383.0,19.0,32.0,20083.0,94.607379,159.338744,3.0
Palma del Condado (La),1187.0,12.0,23.0,10801.0,111.100824,212.943246,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,141.0,8.0,19.0,3092.0,258.732212,614.489004,0.0
San Juan del Puerto,572.0,18.0,48.0,9411.0,191.265540,510.041441,6.0
Beas,232.0,15.0,20.0,4341.0,345.542502,460.723336,0.0
Valverde del Camino,657.0,23.0,51.0,12750.0,180.392157,400.000000,0.0
Calañas,201.0,2.0,10.0,2768.0,72.254335,361.271676,0.0
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0
Lucena del Puerto,267.0,6.0,11.0,3261.0,183.992640,337.319841,0.0
Moguer,1383.0,28.0,70.0,21867.0,128.046829,320.117071,5.0
Huelva (capital),9044.0,176.0,457.0,143837.0,122.360728,317.720753,23.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Niebla,248.0,1.0,9.0,4116.0,24.295432,218.658892,0.0,0.111111
Aracena,539.0,1.0,8.0,8255.0,12.113870,96.910963,0.0,0.125000
Aljaraque,1161.0,7.0,35.0,21474.0,32.597560,162.987799,1.0,0.200000
Ayamonte,1332.0,2.0,10.0,21104.0,9.476876,47.384382,0.0,0.200000
Calañas,201.0,2.0,10.0,2768.0,72.254335,361.271676,0.0,0.200000
Almonaster la Real,69.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0,0.333333
San Juan del Puerto,572.0,18.0,48.0,9411.0,191.265540,510.041441,6.0,0.375000
Sierra de Huelva-Andévalo Central,2917.0,35.0,93.0,67391.0,51.935718,138.000623,0.0,0.376344
